<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_02_Pull_Print_StandAlone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the LICENSE file in the root directory of this source tree.

For those who may not have access to a live MongoDB instance, we have set up a mechanism where the JSON file that would have been loaded into a hosted MongoDB instance, will be loaded into a locally (that is in the VM) installed MongoDB instance and the queries can be run in a manner that is identical to a live, hosted MongoDB instance

![alt text](https://raw.githubusercontent.com/prithwis/parashar21/main/images/weve-moved.jpg) <br>

The Code from this notebook has been moved to the notebook [P21_45_04_YogFilter](https://github.com/prithwis/parashar21/blob/main/P21_45_04_YogFilter.ipynb) 